In [1]:
import pandas as pd
import numpy as np

### Data import

In [4]:
calendar = pd.read_csv('calendar.csv')
prices = pd.read_csv('sell_prices.csv')
train_eval = pd.read_csv('sales_train_evaluation/sales_train_evaluation.csv')

### Combination

In [96]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [86]:
train_eval

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [34]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [255]:
day_col = [col for col in train_eval.columns if col.find('d_')==0]
final_set = pd.DataFrame()
for index in train_eval.index:
    record = train_eval.loc[index]
    #First 6 columns containing basic product information
    non_sales_info = record[:6].to_frame().transpose()
    #Following columns containing product sales information
    sales_info = record[6:].to_frame()
    info = pd.concat([non_sales_info,sales_info],axis=1)
    tmp = info.fillna(method='ffill').drop(index,axis=0).reset_index().rename(columns={'index':'d',0:'sales'})
    tmp = tmp.merge(calendar,left_on='d',right_on='d')
    final_set = final_set.append(tmp,ignore_index=True)
    del tmp

KeyboardInterrupt: 

In [256]:
final_set

,cat_id,d,date,dept_id,event_name_1,event_name_2,event_type_1,event_type_2,id,item_id,...,534,535,536,537,538,539,540,541,542,543
0,HOBBIES,d_1,2011-01-29,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES,d_10,2011-02-07,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES,d_100,2011-05-08,HOBBIES_1,Mother's day,NaN,Cultural,NaN,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES,d_1000,2013-10-24,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES,d_1001,2013-10-25,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055899,HOBBIES,d_995,2013-10-19,HOBBIES_2,NaN,NaN,NaN,NaN,HOBBIES_2_128_CA_1_evaluation,HOBBIES_2_128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1055900,HOBBIES,d_996,2013-10-20,HOBBIES_2,NaN,NaN,NaN,NaN,HOBBIES_2_128_CA_1_evaluation,HOBBIES_2_128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1055901,HOBBIES,d_997,2013-10-21,HOBBIES_2,NaN,NaN,NaN,NaN,HOBBIES_2_128_CA_1_evaluation,HOBBIES_2_128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1055902,HOBBIES,d_998,2013-10-22,HOBBIES_2,NaN,NaN,NaN,NaN,HOBBIES_2_128_CA_1_evaluation,HOBBIES_2_128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [11]:
#Number of rows after transposing our data
train_eval.loc[-28:].shape[0] * train_eval.shape[1]

59364030

In [26]:
#This is the training set containing sales data from day 1 to day 1913
train_set = train_eval.iloc[:,:-28]
#This is the evaluation set containing sales data from day 1914 to day 1941
eval_set = train_eval.iloc[:,-28:]
#Our target is to predict the next 28 days after day 1941(namely to day 1969)